In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.models import load_model
from json import JSONEncoder
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:
train_df0 = pd.read_csv('train.csv')
test_df0 = pd.read_csv('test.csv')
train_df1 = train_df0.append(test_df0,ignore_index=True)
train_df1['rent'] = train_df1['rent'].fillna(0)
train_df2 = train_df1.drop(['id'],axis=1)
train_df2

In [ ]:
# train_df2 = train_df2[[c for c in train_df2 if c not in ['rent']] + ['rent']]
# train_df2

In [ ]:
type_dummies = pd.get_dummies(train_df2['type'])
locality_dummies = pd.get_dummies(train_df2['locality'])
lease_dummies = pd.get_dummies(train_df2['lease_type'])
furnishing_dummies = pd.get_dummies(train_df2['furnishing'])
parking_dummies = pd.get_dummies(train_df2['parking'])
water_dummies = pd.get_dummies(train_df2['water_supply'])
building_dummies = pd.get_dummies(train_df2['building_type'])
facing_dummies = pd.get_dummies(train_df2['facing'])
train_df3 = train_df2.drop(['type','locality','lease_type','furnishing','parking','water_supply','building_type','facing','activation_date','amenities'],axis=1)
train_df4 = pd.concat([type_dummies,locality_dummies,lease_dummies,furnishing_dummies,parking_dummies,water_dummies,building_dummies,facing_dummies,train_df3],axis=1)
train_df4

In [ ]:
train_df3

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train_df4)
scaled_train

In [ ]:
multiplied_by = scaler.scale_[scaled_train.shape[1]-1]
added = scaler.min_[scaled_train.shape[1]-1]
multiplied_by

In [ ]:
scaled_train

In [ ]:
scaled_train_df = pd.DataFrame(scaled_train, columns=train_df4.columns.values)
scaled_train_df

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
target = 'rent'
X = scaled_train_df.drop(target, axis=1).values
Y = scaled_train_df[[target]].values

# Train the model
history = model.fit(
    X[10:20499],
    Y[10:20499],
    epochs=20,
    shuffle=True,
    verbose=2
)

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
prediction = model.predict(X[0:1])
y_0 = prediction[0][0]
print('Prediction with scaling - {}',format(y_0))
y_0 -= added
y_0 /= multiplied_by
print("Housing Price Prediction  - ${}".format(y_0))

In [ ]:
ans_list = []
for i in range(20500,25000):
    prediction = model.predict(X[i:i+1])
    ans = prediction[0][0]
    ans -= added
    ans /= multiplied_by
    ans_list.append(ans)
ans_list

In [ ]:
result = [int(item) for item in ans_list]
df = pd.DataFrame(result,columns=['rent'])
df
df.to_csv('submission.csv',index=False)

In [ ]:
X[:1].shape

In [ ]:
model.save("my_model.h5")

In [ ]:
model2 = load_model('my_model.h5')
multiplied_by

In [ ]:
prediction = model2.predict(X[:1])
print(X[:1].shape)
y_0 = prediction[0][0]
print('Prediction with scaling - {}',format(y_0))
y_0 -= added
y_0 /= multiplied_by
print("Housing Price Prediction  - ${}".format(y_0))

In [ ]:
type(X)

In [ ]:
scaled_train_df

In [ ]:
locality_data = locality_dummies.columns.to_numpy()

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
        
json_data = {"array":locality_data}
with open("locality_array.json", "w") as write_file:
    json.dump(json_data, write_file, cls=NumpyArrayEncoder)


In [ ]:
scaled_train_df['longitude'].mean()

In [ ]:
dict = json.loads(train_df2['amenities'][0])
dict
for ele in dict:
    if dict[ele]==False:
        print(ele)